# Live classification of punch-type and hand

In [41]:
#import neccessary modules
import json
import pandas as pd
import numpy as np

# tsfresh modules (for feature extraction)
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters

import logging
# Set logger-level to "error". Not recommed: Important warnings can be overseen
logging.basicConfig(level=logging.ERROR)

# import machine learning algorithms
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

import pickle

import sys

In [42]:
# helper functions
from timeseries_helpers import datasetstorer
from timeseries_helpers import dataset_importer
from timeseries_helpers import database_importer
from timeseries_helpers import standardizer

## Import trained models (using the best ones)

In [43]:
# Load from file
with open("./Models/model_rndm_forest_clf_punchtype_periodMS2000_sampleUS1000_TrainSize70.pkl", 'rb') as file:  
    model_rndm_forest_clf_punchtype = pickle.load(file)
    
# Load from file
with open("./Models/model_rndm_forest_clf_hand_periodMS2000_sampleUS1000_TrainSize70.pkl", 'rb') as file:  
    model_rndm_forest_clf_hand = pickle.load(file)

In [44]:
with open('./punch_data.json', 'r') as f:
    jsnDataset = json.load(f)

In [47]:
# for debugging: output of input json data (live stream)
print(type(jsnDataset))

In [28]:
#convert dataset to timestamps in absolute us
ds = database_importer.jsonData_to_dataset_in_timedifference_us(data=jsnDataset)

In [29]:
ds_new = standardizer.normate_dataset_period(2000,1000,ds) 
ds = standardizer.change_strings_to_numbs(ds_new)
#del ds['Unnamed: 0'] # remove the unnecessary column ... maybe this can be fixed in a later version of the dataset-exporter?! 

In [30]:
# converts the given list of dataframes to one dataframe
df_list = []
for idx,e in enumerate(ds):
    df = e.assign(punch_id=idx)
    df_list.append(df)
df_res = pd.concat(df_list)
df_new = df_res.reset_index(drop=True)
df_new = df_new.rename(index=str, columns={"x": "a_x", "y": "a_y", "z":"a_z"})

In [33]:
df_res.info()

In [35]:
df_new.info()

In [31]:
# try to use the minimal feature settings
settings_minimal = MinimalFCParameters() # only a few basic features
settings_minimal

In [36]:
# extract features for test dataset
testds_for_extraction = df_new.drop(columns=['label','hand','annotator'])
test_dataset_extracted = extract_features(testds_for_extraction, column_id="punch_id",column_sort="timestamp", impute_function=impute, default_fc_parameters = settings_minimal);

In [37]:
classified_label = model_rndm_forest_clf_punchtype.predict(test_dataset_extracted)
print(classified_label);

In [38]:
classified_hand = model_rndm_forest_clf_hand.predict(test_dataset_extracted)
print(classified_hand);